In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, cross_val_predict, train_test_split, KFold
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, roc_curve, balanced_accuracy_score, accuracy_score, mean_squared_error
from scipy.stats import pearsonr, ttest_rel
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE, Isomap
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
# import xgboost as xgb
import pickle
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import seaborn as sns
# import loralib as lora
import random
# import umap
import re
from itertools import product
import os

# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import math
import random
import numpy as np

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

    

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.head_dim = d_model // nhead

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)

        self.attn_weights = None

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        Q = self.q_linear(query).view(batch_size, -1, self.nhead, self.head_dim).transpose(1, 2)
        K = self.k_linear(key).view(batch_size, -1, self.nhead, self.head_dim).transpose(1, 2)
        V = self.v_linear(value).view(batch_size, -1, self.nhead, self.head_dim).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn = self.dropout(torch.softmax(scores, dim=-1))

        self.attn_weights = attn.detach()
        
        context = torch.matmul(attn, V)

        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.out_linear(context)

        return output

class FeedForward(nn.Module):
    def __init__(self, d_model, dim_feedforward, dropout=0.1):
        super(FeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

    def forward(self, x):
        x = self.dropout(torch.relu(self.linear1(x)))
        x = self.linear2(x)
        return x

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, nhead, dropout)
        self.feed_forward = FeedForward(d_model, dim_feedforward, dropout)
        self.norm1 = nn.Identity()
        self.norm2 = nn.Identity()
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src2 = self.self_attn(src, src, src, mask=src_mask)
        src = src + self.dropout(src2)
        src = self.norm1(src)
        src2 = self.feed_forward(src)
        src = src + self.dropout(src2)
        src = self.norm2(src)
        return src

class TransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout, num_layers):
        super(TransformerEncoder, self).__init__()
        # Create new encoder layer instances directly instead of copying one
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, src, mask=None, src_key_padding_mask=None):
        output = src
        for layer in self.layers:
            output = layer(output, src_mask=mask, src_key_padding_mask=src_key_padding_mask)
        return output
    
class TransformerClassificationNet(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, dim_feedforward, output_dim, dropout=0.1, num_layers_frozen=0):
        super(TransformerClassificationNet, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.num_layers_frozen = num_layers_frozen
        self.transformer_encoder = TransformerEncoder(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            num_layers=num_layers
        )        
        self.decoder = nn.Linear(d_model, output_dim)
        self.d_model = d_model
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)
        self._init_transformer_weights()

    def _init_transformer_weights(self):
        def _init_layer_weights(layer):
            for name, param in layer.named_parameters():
                if param.dim() > 1:
                    nn.init.xavier_uniform_(param.data)
                else:
                    nn.init.uniform_(param.data, -0.1, 0.1)

        for layer in self.transformer_encoder.layers:
            _init_layer_weights(layer)
    def forward(self, src, src_key_padding_mask=None, return_latent=False, return_attentions=False):
        def print_sample_info(tensor, name, num_samples=5, num_values=5):
            print(f"\n{name} shape: {tensor.shape}")
            for i in range(min(num_samples, tensor.shape[0])):
                if tensor.dim() > 1:
                    sample_values = tensor[i, :num_values].tolist()
                    if tensor.dtype in [torch.float32, torch.float64]:
                        sample_mean = tensor[i].mean().item()
                        sample_std = tensor[i].std().item()
                        print(f"Sample {i}: First {num_values} values: {sample_values}")
                        print(f"Sample {i} mean: {sample_mean:.4f}, std: {sample_std:.4f}")
                    else:
                        print(f"Sample {i}: First {num_values} values: {sample_values}")
                else:
                    sample_values = tensor[i].tolist()
                    print(f"Sample {i}: {sample_values}")

        #print_sample_info(src, "Input")

        embedded = self.embedding(src) * math.sqrt(self.embedding.embedding_dim)
        #print_sample_info(embedded, "Embedded")

        pos_encoded = self.pos_encoder(embedded)
        #print_sample_info(pos_encoded, "Pos encoded")

        output = self.transformer_encoder(pos_encoded, src_key_padding_mask=src_key_padding_mask)
        #print_sample_info(output, "Transformer output")
        #output = self.decoder(output[:, -1, :])

        if return_attentions:
            attn_weights = []
            for layer in self.transformer_encoder.layers:
                if hasattr(layer.self_attn, 'attn_weights'):
                    attn_weights.append(layer.self_attn.attn_weights)
        
        #print_sample_info(output, "Mean pooled")
        output = output.mean(dim=1)
        latent = output.clone()
        output = self.decoder(output)
        #print_sample_info(output, "Decoder output")

        final_output = torch.sigmoid(output)
        #print_sample_info(final_output, "Final output")
        if return_latent and return_attentions:
            return final_output, latent, attn_weights
        elif return_latent:
            return final_output, latent
        elif return_attentions:
            return final_output, attn_weights
        else:
            return final_output

    def load_model(self, path):
        state_dict = torch.load(path)
        model_dict = self.state_dict()
        
        # print("Keys in state_dict (pretrained model):")
        # print(sorted(state_dict.keys()))
        # print("\nKeys in model_dict (current model):")
        # print(sorted(model_dict.keys()))
        
        # Filter out unnecessary keys and handle mismatches
        new_state_dict = {}
        for key, value in state_dict.items():
            if key in model_dict:
                if model_dict[key].shape == value.shape:
                    new_state_dict[key] = value                
        
        # Print out any missing or unexpected keys
        missing_keys = set(model_dict.keys()) - set(new_state_dict.keys())
        unexpected_keys = set(state_dict.keys()) - set(model_dict.keys())
        
        if missing_keys:
            print(f"Missing keys in loaded model: {missing_keys}")
        if unexpected_keys:
            print(f"Unexpected keys in loaded model: {unexpected_keys}")
        
        # Load the filtered state dict
        self.load_state_dict(new_state_dict, strict=False)
            
    def freeze_transformer(self):
        """
        Freezes the first num_layers_frozen layers of the transformer,
        along with embeddings and positional encodings if any layers are frozen.
        Assumes 4-layer transformer.
        """
        if self.num_layers_frozen > 0:
            # If we're freezing any layers, also freeze embeddings and positional encodings
            for param in self.embedding.parameters():
                param.requires_grad = False
            for param in self.pos_encoder.parameters():
                param.requires_grad = False
            
            # Freeze the specified number of transformer layers from the bottom
            for i, layer in enumerate(self.transformer_encoder.layers):
                if i < self.num_layers_frozen:  # Freeze early layers
                    for param in layer.parameters():
                        param.requires_grad = False
                else:  # Keep later layers trainable
                    for param in layer.parameters():
                        param.requires_grad = True
                        
        # Print freezing status for verification
        for name, param in self.named_parameters():
            if param.requires_grad:
                print(f"Trainable: {name}")
            else:
                print(f"Frozen: {name}")


class PatientDataset(Dataset):
    def __init__(self, data, labels, lengths):
        self.data = data
        self.labels = labels
        self.lengths = lengths

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx], self.lengths[idx]

class EHRDataset(Dataset):
    def __init__(self, codes, outcomes, lengths):
        self.codes = codes
        self.outcomes = torch.tensor(outcomes, dtype=torch.float32)  # Convert to tensor here
        self.lengths = lengths

    def __len__(self):
        return len(self.outcomes)

    def __getitem__(self, idx):
        return self.codes[idx], self.outcomes[idx], self.lengths[idx]

def create_attention_mask(lengths, max_len):
    device = lengths.device
    mask = (torch.arange(max_len, device=device)[None, :] < lengths[:, None]).to(device)
    return mask.transpose(0, 1)  # Transpose the mask to match the expected shape

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def collate_fn(batch):
    if len(batch[0]) == 3:  # If proteomics data is included
        data, labels, proteomics = zip(*batch)
        data = pad_sequence([torch.LongTensor(d) for d in data], batch_first=True, padding_value=0)
        return data, torch.tensor(labels, dtype=torch.float32), torch.tensor(proteomics, dtype=torch.float32)
    else:
        data, labels = zip(*batch)
        data = pad_sequence([torch.LongTensor(d) for d in data], batch_first=True, padding_value=0)
        return data, torch.tensor(labels, dtype=torch.float32)

def create_dataloader(EHR_codes, proteomics, outcomes, lengths, batch_size, feature_types):
    dataset = EHRDataset(EHR_codes, outcomes, lengths)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

def normalize_dataset(dataset):
    for i, sequence in enumerate(dataset):
        dataset[i] = (sequence - sequence.mean(dim=0, keepdim=True)) / (sequence.std(dim=0, keepdim=True) + 1e-8)
    return dataset

def impute_missing_values(dataset):
    # Stack all tensors in the dataset along a new dimension, creating a tensor of shape (num_samples, max_seq_length, num_features)
    stacked_data = torch.stack(dataset)

    # Calculate the mean of each feature across all samples and sequences, ignoring NaN values
    feature_means = torch.nanmean(stacked_data, dim=(0, 1))

    # Iterate through the dataset (list of tensors)
    for i, sequence in enumerate(dataset):
        # Create a boolean mask indicating the positions of NaN values in the sequence
        mask = torch.isnan(sequence)

        # Replace NaN values in the sequence with the corresponding feature means
        # 'expand_as' is used to match the dimensions of the mask and the sequence
        dataset[i][mask] = feature_means.expand_as(sequence)[mask]

    return dataset
        
def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)

In [ ]:
class DataBuilder(Dataset):
    def __init__(self, x, y, standardizer):
        self.x, self.y, self.standardizer = x, y, standardizer
        self.len=self.x.shape[0]
    def __getitem__(self,index):      
        return (self.x[index], self.y[index])
    def __len__(self):
        return self.len

In [ ]:
#for this project, feature_types is always EHR
def run_experiment(EHR_tokens, proteomics, patient_indices, outcomes, lengths, experiment_name, lr, lr_decay,
                   bs, prot_hidden_dim=32, train_indices=None, val_indices=None, test_indices=None, feature_types='EHR', model_path='', fine_tune=False, seed=42, num_layers=2, hidden_dim=400,
                   dropout=0.4, return_preds=False, return_interpretability=False, return_grads=False,
                   protein_weights=None, blend=None, hyperparam_tuning=False, vocab_size=None, num_frozen=0):
    
    set_seed(seed)
    prediction_module_hidden_sizes = [hidden_dim, hidden_dim//2, hidden_dim//4, hidden_dim//8]
    
    assert feature_types in ['EHR', 'metab', 'both']   
    if feature_types == 'metab': assert model_path == ''
    if (model_path != '') & (feature_types == 'both'): assert fine_tune==True
    if hyperparam_tuning == False: assert train_indices == None

    if hyperparam_tuning == False:
        maternal_IDs = patient_indices['person_id']
        train_ratio, test_ratio, val_ratio = 0.70, 0.15, 0.15
        train_ids, temp_ids = train_test_split(maternal_IDs, test_size=(test_ratio + val_ratio), random_state=seed, stratify=outcomes)
        patient_indices_temp = patient_indices.copy(deep=True)
        patient_indices_temp.index = patient_indices_temp['person_id']
        temp_indices = patient_indices_temp.loc[temp_ids, 'index'].values
        test_ids, val_ids = train_test_split(temp_ids, test_size=(val_ratio / (test_ratio + val_ratio)), random_state=seed, stratify=outcomes[temp_indices])
        
        if feature_types != 'EHR':
            proteomics = proteomics.merge(patient_indices[['person_id','index']], how='left', left_on='person_id', right_on='person_id').drop(['person_id'],axis=1).sort_values('index').drop('index',axis=1).values

        train_indices = patient_indices[patient_indices['person_id'].isin(train_ids)]['index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices[patient_indices['person_id'].isin(test_ids)]['index'].values
        val_indices = patient_indices[patient_indices['person_id'].isin(val_ids)]['index'].values

    # Prepare data
    train_EHR_tokens = EHR_tokens[train_indices, :]
    test_EHR_tokens = EHR_tokens[test_indices, :]
    val_EHR_tokens = EHR_tokens[val_indices, :]
    
    if feature_types != 'EHR':
        train_proteomics = proteomics[train_indices, :]
        test_proteomics = proteomics[test_indices, :]
        val_proteomics = proteomics[val_indices, :]
        scaler = StandardScaler()
        train_proteomics = scaler.fit_transform(train_proteomics)
        test_proteomics = scaler.transform(test_proteomics)
        val_proteomics = scaler.transform(val_proteomics)
    else:
        train_proteomics = test_proteomics = val_proteomics = None

    train_outcomes = outcomes[train_indices]
    test_outcomes = outcomes[test_indices]
    val_outcomes = outcomes[val_indices]

    train_lengths = lengths[train_indices]
    test_lengths = lengths[test_indices]
    val_lengths = lengths[val_indices]

    # Create dataloaders
    train_loader = create_dataloader(train_EHR_tokens, train_proteomics, train_outcomes, train_lengths, bs, feature_types)
    test_loader = create_dataloader(test_EHR_tokens, test_proteomics, test_outcomes, test_lengths, bs, feature_types)
    val_loader = create_dataloader(val_EHR_tokens, val_proteomics, val_outcomes, val_lengths, bs, feature_types)

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    # Model initialization
    if vocab_size == None:
        vocab_size = EHR_tokens.max().item() + 1
    if model_path == '':
        if feature_types == 'EHR':
            print(f"Initializing TransformerClassificationNet with parameters:")
            print(f"vocab_size: {vocab_size}, d_model: {hidden_dim}, nhead: 4, num_layers: {num_layers}, dim_feedforward: {hidden_dim*4}, output_size: 1, dropout: {dropout}")
            try:
                model = TransformerClassificationNet(
                    vocab_size=vocab_size, 
                    d_model=hidden_dim, 
                    nhead=4,
                    num_layers=num_layers, 
                    dim_feedforward=hidden_dim * 4,
                    output_dim=1,
                    dropout=dropout,
                    num_layers_frozen=num_frozen
                )
                model.to(device)
                if num_frozen > 0:
                    model.freeze_transformer()
            except RuntimeError as e:
                print(f"Error initializing model: {str(e)}")
                print("CUDA error: device-side assert triggered. This might be due to incompatible dimensions or insufficient GPU memory.")
                print(f"EHR_tokens shape: {EHR_tokens.shape}")
                print(f"Available GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
                print(f"Current GPU memory usage: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
                raise
        elif feature_types == 'metab':
            model = proteomics_net(proteomics.shape[1], [prot_hidden_dim], 1, dropout).to(device)
        elif feature_types == 'both':
            model = JointTransformerClassification(
                vocab_size=vocab_size,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                output_size=1,
                input_size_proteomics=proteomics.shape[1],
                proteomics_hidden_layers=[prot_hidden_dim],
                nhead=4,
                dropout=dropout
            ).to(device)
    else:
        if feature_types == 'EHR':
            model = TransformerClassificationNet(
                vocab_size=vocab_size, 
                d_model=hidden_dim, 
                nhead=4,
                num_layers=num_layers, 
                dim_feedforward=hidden_dim * 4,
                output_dim=1,
                dropout=dropout,
                num_layers_frozen=num_frozen
            )
            model.load_model(model_path)
            model.to(device)
            if num_frozen > 0:
                model.freeze_transformer()
            if not fine_tune:
                model.eval()
                return evaluate_model(model, val_loader, device, return_preds)
        elif feature_types == 'both':
            model = JointTransformerClassification(
                vocab_size=vocab_size,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                output_size=1,
                input_size_proteomics=proteomics.shape[1],
                proteomics_hidden_layers=[prot_hidden_dim],
                nhead=4,
                dropout=dropout
            )
            model.load_model(model_path)
            model.to(device)
            if fine_tune:
                model.freeze_transformer()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=lr_decay)
    num_epochs = 100
    train_losses, test_losses, val_losses = [], [], []
    best_loss = np.inf
    patience, counter = 5, 0

    for epoch in tqdm(range(num_epochs)):
        model.train()
        if not hyperparam_tuning:
            torch.save(model.state_dict(), f'./models/predictive_models/{experiment_name}_epoch{epoch}.pth')
        
        train_loss = train_epoch(model, train_loader, optimizer, criterion, device, feature_types, blend, PT_EHR_model if blend else None)
        train_losses.append(train_loss)

        model.eval()
        test_loss, predictions, true_labels = evaluate_epoch(model, test_loader, criterion, device, feature_types)
        test_losses.append(test_loss)

        pearson_corr = roc_auc_score(true_labels, predictions)
        print(f'Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test AUROC: {pearson_corr:.4f}')
        if test_loss < best_loss:
            best_loss = test_loss
            counter = 0
            torch.save(model.state_dict(), f'./models/predictive_models/{experiment_name}.pth')
        else:
            counter += 1
            if counter >= patience:
                break

    model.load_state_dict(torch.load(f'./models/predictive_models/{experiment_name}.pth'))
    val_loss, val_predictions, val_true_labels = evaluate_epoch(model, val_loader, criterion, device, feature_types)
    val_losses.append(val_loss)
    pearson_corr = roc_auc_score(val_true_labels, val_predictions)

    print(f'Epoch: {epoch+1}/{num_epochs}, Val Loss: {val_loss:.4f}, AUC: {pearson_corr:.4f}')

    if hyperparam_tuning:
        os.remove(f'./models/predictive_models/{experiment_name}.pth')

    if return_preds:
        return pearson_corr, val_loss, None, val_outcomes, np.array(val_predictions), val_indices
    else:
        return pearson_corr, val_loss, None

def train_epoch(model, loader, optimizer, criterion, device, feature_types, blend, PT_EHR_model):
    running_loss, num_samples = 0, 0
    for batch_idx, batch in enumerate(loader):
        if feature_types == 'EHR':
            tokens, labels, lengths = batch
            tokens, labels, lengths = tokens.to(device), labels.to(device), lengths.to(device)
            proteomics = None
        elif feature_types == 'metab':
            proteomics, labels = batch
            proteomics, labels = proteomics.to(device), labels.to(device)
        else:
            tokens, proteomics, labels, lengths = batch
            tokens, proteomics, labels, lengths = tokens.to(device), proteomics.to(device), labels.to(device), lengths.to(device)

        optimizer.zero_grad()

        if blend:
            with torch.no_grad():
                _, (optimal_latent, _, _, _, _, _) = PT_EHR_model(tokens, None, lengths, interpretability=True)
            outputs = model(tokens, proteomics, lengths, better_latent=optimal_latent, better_ratio=0.5)
        else:
            if feature_types == 'EHR':
                outputs = model(tokens, lengths)
            elif feature_types == 'metab':
                proteomics = proteomics.float()
                outputs = model(proteomics)
            else:  # 'both'
                proteomics = proteomics.float()
                outputs = model(tokens, proteomics, lengths)

        loss = criterion(outputs.squeeze(-1), labels)  # Only squeeze the last dimension
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        running_loss += loss.item() * labels.size(0)
        num_samples += labels.size(0)

    return running_loss / num_samples

def evaluate_epoch(model, loader, criterion, device, feature_types):
    running_loss, num_samples = 0, 0
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in loader:
            if feature_types == 'EHR':
                tokens, labels, lengths = batch
                tokens, labels, lengths = tokens.to(device), labels.to(device), lengths.to(device)
                proteomics = None
            elif feature_types == 'metab':
                proteomics, labels = batch
                proteomics, labels = proteomics.to(device), labels.to(device)
            else:
                tokens, proteomics, labels, lengths = batch
                tokens, proteomics, labels, lengths = tokens.to(device), proteomics.to(device), labels.to(device), lengths.to(device)

            if feature_types == 'EHR':
                outputs = model(tokens, lengths)
            elif feature_types == 'metab':
                proteomics = proteomics.float()
                outputs = model(proteomics)
            else:  # 'both'
                proteomics = proteomics.float()
                outputs = model(tokens, proteomics, lengths)

            loss = criterion(outputs.squeeze(-1), labels)  # Only squeeze the last dimension


            running_loss += loss.item() * labels.size(0)
            num_samples += labels.size(0)
            predictions.extend(outputs.squeeze(-1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    return running_loss / num_samples, predictions, true_labels

def evaluate_model(model, loader, device, return_preds):
    criterion = nn.BCELoss()
    val_loss, val_predictions, val_true_labels = evaluate_epoch(model, loader, criterion, device, 'EHR')
    pearson_corr = roc_auc_score(val_true_labels, val_predictions)
    print(f'Validation Loss: {val_loss:.4f}, AUC: {pearson_corr:.4f}')
    if return_preds:
        return pearson_corr, val_loss, None, val_true_labels, val_predictions, None
    else:
        return pearson_corr, val_loss, None

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)

In [ ]:
overall_best_params = {}

In [ ]:
outcome_list = ['MACE']

In [ ]:
NUM_TRIALS = 25

In [ ]:
patient_indices_PT = pd.read_csv('./processed_data/transformer_sample_id_to_index.csv')


In [ ]:
# Load pre-training cohort data
transformer_input_sequences_PT = np.load('./processed_data/transformer_input_sequences.npy')
transformer_input_lengths_PT = np.load('./processed_data/transformer_input_lengths.npy')
patient_indices_PT = pd.read_csv('./processed_data/transformer_sample_id_to_index.csv')
transformer_outcomes_PT = np.load('./processed_data/transformer_mace_outcomes.npy')

print("PT cohort data shapes:")
print(f"Input sequences: {transformer_input_sequences_PT.shape}")
print(f"Input lengths: {transformer_input_lengths_PT.shape}")
print(f"Outcomes: {transformer_outcomes_PT.shape}")
print(f"Patient indices: {patient_indices_PT.shape}")

# Load omics cohort data
transformer_input_sequences_omics = np.load('./processed_data/transformer_input_sequences_FT.npy')
transformer_input_lengths_omics = np.load('./processed_data/transformer_input_lengths_FT.npy')
patient_indices_omics = pd.read_csv('./processed_data/transformer_sample_id_to_index_FT.csv')
transformer_outcomes_omics = np.load('./processed_data/transformer_mace_outcomes_FT.npy')

print("\nOmics cohort data shapes:")
print(f"Input sequences: {transformer_input_sequences_omics.shape}")
print(f"Input lengths: {transformer_input_lengths_omics.shape}")
print(f"Outcomes: {transformer_outcomes_omics.shape}")
print(f"Patient indices: {patient_indices_omics.shape}")

# Load vocabularies
with open('./processed_data/transformer_vocab.pkl', 'rb') as f:
    vocab_PT = pickle.load(f)

with open('./processed_data/transformer_vocab_FT.pkl', 'rb') as f:
    vocab_omics = pickle.load(f)

print(f"\nPT Vocabulary size: {len(vocab_PT)}")
print(f"Omics Vocabulary size: {len(vocab_omics)}")
# Create a new merged vocabulary that preserves IDs from both original vocabularies
merged_vocab = {}
current_max_id = 0

# First, add all tokens from the PT vocabulary
for token, idx in vocab_PT.items():
    merged_vocab[token] = idx
    current_max_id = max(current_max_id, idx)

# Then, add tokens from the omics vocabulary, preserving their IDs if not already present
for token, idx in vocab_omics.items():
    if token not in merged_vocab:
        merged_vocab[token] = idx
    current_max_id = max(current_max_id, idx)

# Add any new tokens that might have different IDs in omics vocab
for token, idx in vocab_omics.items():
    if token not in merged_vocab:
        current_max_id += 1
        merged_vocab[token] = current_max_id

print(f"Merged Vocabulary size: {len(merged_vocab)}")

# Function to update token IDs based on merged vocabulary
def update_token_ids(sequences, old_vocab, new_vocab):
    old_vocab_reverse = {v: k for k, v in old_vocab.items()}
    updated_sequences = np.zeros_like(sequences)
    for i, seq in tqdm(enumerate(sequences), total=len(sequences)):
        updated_seq = []
        for token_id in seq:
            if token_id in old_vocab_reverse:
                token = old_vocab_reverse[token_id]
                updated_seq.append(new_vocab[token])
            else:
                updated_seq.append(new_vocab['[PAD]'])
        updated_sequences[i] = updated_seq
    return updated_sequences

# Update token IDs in both datasets
transformer_input_sequences_PT = update_token_ids(transformer_input_sequences_PT, vocab_PT, merged_vocab)
transformer_input_sequences_omics = update_token_ids(transformer_input_sequences_omics, vocab_omics, merged_vocab)

print("\nUpdated data shapes:")
print(f"PT input sequences: {transformer_input_sequences_PT.shape}")
print(f"Omics input sequences: {transformer_input_sequences_omics.shape}")

# Verification
print("\nVerification:")
print(f"Number of unique tokens in PT sequences: {len(np.unique(transformer_input_sequences_PT))}")
print(f"Number of unique tokens in Omics sequences: {len(np.unique(transformer_input_sequences_omics))}")
print(f"Max token ID in PT sequences: {np.max(transformer_input_sequences_PT)}")
print(f"Max token ID in Omics sequences: {np.max(transformer_input_sequences_omics)}")

# Check for ID consistency
pt_sample = transformer_input_sequences_PT[0][:10]  # First 10 tokens of first sequence
omics_sample = transformer_input_sequences_omics[0][:10]  # First 10 tokens of first sequence

print("\nSample token IDs:")
print(f"PT: {pt_sample}")
print(f"Omics: {omics_sample}")
merged_vocab_reverse = {v: k for k, v in merged_vocab.items()}

# Decode samples using the merged vocabulary
print("\nDecoded PT sample with merged vocab:")
print([merged_vocab_reverse.get(token_id, '[UNK]') for token_id in pt_sample])

print("\nDecoded Omics sample with merged vocab:")
print([merged_vocab_reverse.get(token_id, '[UNK]') for token_id in omics_sample])


In [ ]:
np.sum(transformer_outcomes_PT)/len(transformer_outcomes_PT)

In [ ]:
np.sum(transformer_outcomes_omics)/len(transformer_outcomes_omics)

In [ ]:
import torch

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory Allocated: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
        print(f"  Memory Cached:    {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB")
else:
    print("No CUDA-enabled GPU found.")


### Run cell below if loading old hyperparams

In [ ]:
overall_best_params = pickle.load(open('best_hyperparams_transformer.pkl','rb'))


In [ ]:
overall_best_params

### Hyperparam Tuning

In [ ]:
#Testing parameters: {'batch_size': 128, 'lr': 0.001, 'dropout': 0.3, 'lr_decay': 0.001, 'layers': 4, 'hidden_dim': 256}

import numpy as np
from itertools import product
from tqdm.notebook import tqdm
import pandas as pd
import pickle
import os

# Hyperparameter grid
grid_search = {
    'batch_size': [128],
    'lr': [1e-3, 1e-4],
    'dropout': [0.1, 0.3, 0.5],
    'lr_decay': [1e-4, 1e-3],
    'layers': [1,2,4],
    'hidden_dim': [128, 256]
}

# Generate all combinations of hyperparameters
all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

# Create or load results file
results_file = "hyperparam_search_results.csv"
if os.path.exists(results_file):
    results_df = pd.read_csv(results_file)
    print(f"Loaded {len(results_df)} existing results")
else:
    results_df = pd.DataFrame(columns=[
        'outcome', 'num_layers', 'dropout', 'lr', 'lr_decay', 
        'hidden_dim', 'batch_size', 'val_loss', 'val_auroc'  # Added val_auroc
    ])

# Main loop
for i in tqdm(outcome_list):
    overall_best_params[i] = {'PT': {}}
    maternal_IDs = patient_indices_PT['person_id'].unique()
    
    # Subsample the data
    subsample_size = int(0.01 * len(maternal_IDs))
    subsampled_IDs = np.random.choice(maternal_IDs, subsample_size, replace=False, p=None)
    
    # Single train/val/test split
    train_size = int(0.7 * len(subsampled_IDs))
    val_size = int(0.15 * len(subsampled_IDs))
    
    # Split IDs
    train_IDs = subsampled_IDs[:train_size]
    val_IDs = subsampled_IDs[train_size:train_size + val_size]
    test_IDs = subsampled_IDs[train_size + val_size:]
    
    # Get indices
    train_indices = patient_indices_PT[patient_indices_PT['person_id'].isin(train_IDs)]['index'].values
    test_indices = patient_indices_PT[patient_indices_PT['person_id'].isin(test_IDs)]['index'].values
    val_indices = patient_indices_PT[patient_indices_PT['person_id'].isin(val_IDs)]['index'].values
    np.random.shuffle(train_indices)
    
    print(f"Dataset sizes after subsample:")
    print(f"Total samples: {len(subsampled_IDs)}")
    print(f"Train: {len(train_indices)}, Val: {len(val_indices)}, Test: {len(test_indices)}")
    
    # Test all parameter combinations
    for param_set in tqdm(all_params):
        # Check if we've already tested these parameters for this outcome
        param_match = (results_df['outcome'] == i)
        for key, value in param_set.items():
            if key == 'batch_size':
                continue  # Skip batch_size as it's constant
            # Map 'layers' to 'num_layers' for DataFrame comparison
            df_key = 'num_layers' if key == 'layers' else key
            param_match &= (results_df[df_key] == value)
            
        if param_match.any():
            print(f"Skipping parameter set - already tested")
            continue
            
        bs = param_set['batch_size']
        lr = param_set['lr']
        dropout = param_set['dropout']
        lr_decay = param_set['lr_decay']
        layers = param_set['layers']
        hidden_dim = param_set['hidden_dim']
        
        model_name = f'PT_MODEL_{layers}_{lr}_{lr_decay}_{dropout}_{hidden_dim}'
        print(f"Testing parameters: {param_set}")
        
        try:
            val_auroc, val_loss, _ = run_experiment(  # Now correctly capturing the AUROC from first return value
                transformer_input_sequences_PT, 
                None,
                patient_indices_PT, 
                transformer_outcomes_PT, 
                transformer_input_lengths_PT, 
                model_name,
                lr, 
                lr_decay, 
                bs,
                train_indices=train_indices,
                test_indices=test_indices,
                val_indices=val_indices,
                feature_types='EHR',
                model_path='',
                fine_tune=False,
                seed=42,
                hidden_dim=hidden_dim,
                num_layers=layers,
                dropout=dropout,
                hyperparam_tuning=True,
                vocab_size=len(merged_vocab)
            )
            
            # Add result to DataFrame
            new_result = pd.DataFrame({
                'outcome': [i],
                'num_layers': [layers],
                'dropout': [dropout],
                'lr': [lr],
                'lr_decay': [lr_decay],
                'hidden_dim': [hidden_dim],
                'batch_size': [bs],
                'val_loss': [val_loss],
                'val_auroc': [val_auroc]  # Added val_auroc from first return value
            })
            
            results_df = pd.concat([results_df, new_result], ignore_index=True)
            
            # Save after each experiment
            results_df.to_csv(results_file, index=False)
            
        except Exception as e:
            print(f"Error with parameters {param_set}: {str(e)}")
            continue
    
    print(f'PT\noutcome {i}')

    # Find best parameters for this outcome
    outcome_results = results_df[results_df['outcome'] == i]
    best_row = outcome_results.loc[outcome_results['val_loss'].idxmin()]
    
    model_name = f'PT_MODEL_{best_row["num_layers"]}_{best_row["lr"]}_{best_row["lr_decay"]}_{best_row["dropout"]}_{best_row["hidden_dim"]}'
    
    overall_best_params[i]['PT'] = {
        'num_layers': int(best_row['num_layers']),
        'lr': best_row['lr'],
        'lr_decay': best_row['lr_decay'],
        'dropout': best_row['dropout'],
        'hidden_dim': int(best_row['hidden_dim']),
        'batch_size': int(best_row['batch_size']),
        'model_name': model_name
    }
    
    # Save best parameters after each outcome
    with open("best_hyperparams_transformer.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
# hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
# hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
# hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
# num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
# print(np.min(hyperparam_df['val_loss']))
# model_name = 'PT_MODEL_{}_{}_{}_{}_{}'.format(num_layers,lr,lr_decay,dropout, hidden_dim)
# overall_best_params['stroke']['PT'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
#                                 'hidden_dim': int(hidden_dim), 'batch_size': int(bs), 'model_name': model_name}
# with open("best_hyperparams_transformer.pkl", "wb") as f:
#         pickle.dump(overall_best_params, f)

In [ ]:
best_num_layers = overall_best_params['MACE']['PT']['num_layers']
best_dropout = overall_best_params['MACE']['PT']['dropout']
best_model_name = overall_best_params['MACE']['PT']['model_name']
best_hidden_dim = overall_best_params['MACE']['PT']['hidden_dim']
best_num_layers, best_dropout, best_hidden_dim, best_model_name

In [ ]:
overall_best_params

In [ ]:
%%time
val_r, val_loss, val_rmse = run_experiment(transformer_input_sequences_PT, None,
            patient_indices_PT, transformer_outcomes_PT, transformer_input_lengths_PT, overall_best_params['MACE']['PT']['model_name'], 
    overall_best_params['MACE']['PT']['lr'], overall_best_params['MACE']['PT']['lr_decay'],
       128, feature_types='EHR', model_path='', fine_tune=False, seed=3, 
    hidden_dim=overall_best_params['MACE']['PT']['hidden_dim'], num_layers=overall_best_params['MACE']['PT']['num_layers'], dropout=overall_best_params['stroke']['PT']['dropout'], hyperparam_tuning=False, vocab_size=len(merged_vocab))

In [ ]:
grid_search = {
    'batch_size': [16],
    'lr': [1e-2,1e-3, 1e-4],
    'dropout': [0.1, 0.3, 0.5],
    'lr_decay': [1e-4, 1e-3],
    'layers': [1,2, 4],
    'hidden_dim': [128, 256]
}

all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
# Assume that grid_search and all_params are already defined
# all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp1'] = {}

    IDs = patient_indices_omics['person_id'].unique()
    
    kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in kf.split(IDs, transformer_outcomes_omics):
        split_num += 1
        results_dict = {}
        train_IDs = IDs[train_index]
        test_IDs = IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        


        train_indices = patient_indices_omics[patient_indices_omics['person_id'].isin(train_IDs)]['index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_omics[patient_indices_omics['person_id'].isin(test_IDs)]['index'].values
        val_indices = patient_indices_omics[patient_indices_omics['person_id'].isin(val_IDs)]['index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = param_set['dropout']
            lr_decay = param_set['lr_decay']
            layers = param_set['layers']
            hidden_dim = param_set['hidden_dim']
            print(param_set)
            val_r, val_loss, val_rmse = run_experiment(transformer_input_sequences_omics, None,
                                                       patient_indices_omics, transformer_outcomes_omics,
                                                       transformer_input_lengths_omics, 'EHR_omics_only',
                                                       lr, lr_decay, bs, 
                                                       train_indices=train_indices, test_indices=test_indices,
                                                       val_indices=val_indices, feature_types='EHR', model_path='',
                                                       fine_tune=False, seed=42, hidden_dim=hidden_dim,
                                                       num_layers=layers, dropout=dropout, hyperparam_tuning=True)

            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)

        print('experiment 1')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))

print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)

overall_best_params['MACE']['exp1'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,'hidden_dim': int(hidden_dim), 'batch_size': int(bs)}
with open("./best_hyperparams_transformer.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

In [ ]:
for i in tqdm(outcome_list):
    overall_best_params[i]['exp4'] = {}
    results_dict = {}

    # Iterate through the hyperparameter combinations
    bs = 1000
    lr = 0.1
    dropout = best_dropout
    lr_decay = 0.1
    layers = best_num_layers
    hidden_dim = best_hidden_dim
    # print(param_set)
    val_r, val_loss, val_rmse = run_experiment(transformer_input_sequences_omics, None,
            patient_indices_omics, transformer_outcomes_omics, transformer_input_lengths_omics, 'EHR_omics_PT', 
            lr, lr_decay, bs, feature_types='EHR', model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                                               fine_tune=False, seed=42,
                                               hidden_dim=hidden_dim,
                                              num_layers=layers, dropout=dropout, hyperparam_tuning=False, vocab_size = len(merged_vocab))

    results_dict[val_loss] = {'num_layers': layers,'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                            'hidden_dim': hidden_dim, 'batch_size': bs}

    print('experiment 4')
    print('outcome {}'.format(i))
    overall_best_params[i]['exp4'] = results_dict[min(results_dict.keys())]
    print(results_dict[min(results_dict.keys())])

# Save the best hyperparameters to a pickle file
with open("./best_hyperparams_transformer.pkl", "wb") as f:
    pickle.dump(overall_best_params, f)

In [ ]:
grid_search = {
    'batch_size': [16],
    'lr': [1e-2,1e-3, 1e-4],
    'dropout': [0.1, 0.3, 0.5],
    'lr_decay': [1e-4, 1e-3],
    'num_frozen':range(0, best_num_layers+1)}


all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

In [ ]:
# Assume that grid_search and all_params are already defined
# all_params = [dict(zip(grid_search.keys(), values)) for values in product(*grid_search.values())]

num_layers_arr = []
dropout_arr = []
lr_arr = []
lr_decay_arr = []
hidden_dim_arr = []
batch_size_arr = []
split_num_arr = []
loss_arr = []
num_frozen_arr = []
for i in tqdm(outcome_list):
    overall_best_params[i]['exp5'] = {}

    IDs = patient_indices_omics['person_id'].unique()
    
    kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    split_num = 0
    for train_index, test_index in kf.split(IDs, transformer_outcomes_omics):
        split_num += 1
        results_dict = {}
        train_IDs = IDs[train_index]
        test_IDs = IDs[test_index]
        
        sample_size = int(0.2 * len(train_index))
        random_indices = np.random.choice(train_IDs.shape[0], sample_size, replace=False)
        val_IDs = train_IDs[random_indices]
        train_IDs = np.delete(train_IDs, random_indices)
        

        train_indices = patient_indices_omics[patient_indices_omics['person_id'].isin(train_IDs)]['index'].values
        np.random.shuffle(train_indices)
        test_indices = patient_indices_omics[patient_indices_omics['person_id'].isin(test_IDs)]['index'].values
        val_indices = patient_indices_omics[patient_indices_omics['person_id'].isin(val_IDs)]['index'].values

        # Iterate through the hyperparameter combinations
        for param_set in tqdm(all_params):
            bs = param_set['batch_size']
            lr = param_set['lr']
            dropout = best_dropout
            lr_decay = param_set['lr_decay']
            layers = best_num_layers
            hidden_dim = best_hidden_dim
            num_frozen = param_set['num_frozen']
            print(param_set)
            
            val_r, val_loss, val_rmse = run_experiment(transformer_input_sequences_omics, None,
            patient_indices_omics, transformer_outcomes_omics, transformer_input_lengths_omics, 'EHR_omics_PT_FT',
                lr, lr_decay, bs, train_indices=train_indices, test_indices=test_indices,
                val_indices=val_indices, feature_types='EHR',
                model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                fine_tune=True, seed=42, hidden_dim=hidden_dim, num_layers=layers, dropout=dropout,
                                                       hyperparam_tuning=True, vocab_size=len(merged_vocab), num_frozen=num_frozen)
            
            num_layers_arr.append(layers)
            dropout_arr.append(dropout)
            lr_arr.append(lr)
            lr_decay_arr.append(lr_decay)
            hidden_dim_arr.append(hidden_dim)
            batch_size_arr.append(bs)
            split_num_arr.append(split_num)
            loss_arr.append(val_loss)
            num_frozen_arr.append(num_frozen)

        print('experiment 5')
        print('outcome {}'.format(i))

In [ ]:
hyperparam_df = pd.DataFrame([num_layers_arr, dropout_arr, lr_arr, lr_decay_arr, hidden_dim_arr, batch_size_arr, num_frozen_arr,split_num_arr,loss_arr]).T
hyperparam_df.columns = ['num_layers','dropout','lr','lr_decay','hidden_dim','bs','num_frozen','split_num','val_loss']
hyperparam_df = hyperparam_df.groupby(['num_layers','dropout','lr','lr_decay','hidden_dim','bs','num_frozen']).mean()
num_layers, dropout, lr, lr_decay, hidden_dim, bs,num_frozen = hyperparam_df['val_loss'].idxmin()
print(np.min(hyperparam_df['val_loss']))
print(num_layers, dropout, lr, lr_decay, hidden_dim, bs)

overall_best_params['MACE']['exp5'] = {'num_layers': int(num_layers),'lr': lr,'lr_decay': lr_decay, 'dropout': dropout,
                                'hidden_dim': int(hidden_dim), 'batch_size': int(bs),'num_frozen': int(num_frozen)}
with open("./best_hyperparams_transformer.pkl", "wb") as f:
        pickle.dump(overall_best_params, f)

### Modeling

In [ ]:
overall_best_params

In [ ]:
#experiment 1 = CB model EHR features
#experiment 2 = CB model metab features
#experiment 3 = CB model all features
#experiment 4 = only pretrained model
#experiment 5 = fine tune pretrained model 
#experiment 6 = add metabs, fine tune pretrained model

num_iterations = 50
results = {}
for i in tqdm(outcome_list):
    results[i] = {'exp1': [],'exp5':[]}
    for j in tqdm(range(0, num_iterations)):
        print('experiment 1')
        val_auc = run_experiment(transformer_input_sequences_omics, None,
            patient_indices_omics, transformer_outcomes_omics, transformer_input_lengths_omics, 'EHR_omics_only_{}'.format(j), 
                overall_best_params[i]['exp1']['lr'], overall_best_params[i]['exp1']['lr_decay'],
                overall_best_params[i]['exp1']['batch_size'], feature_types='EHR', model_path='', fine_tune=False, seed=j,
                                hidden_dim=overall_best_params[i]['exp1']['hidden_dim'],
                                 num_layers=overall_best_params[i]['exp1']['num_layers'],
                                 dropout=overall_best_params[i]['exp1']['dropout'], return_preds=True)
        results[i]['exp1'].append(val_auc)
        
        # print('experiment 2')
        # val_auc = run_experiment(transformer_input_sequences_omics, proteomics,
        #     patient_indices, transformer_outcomes_omics, transformer_input_lengths_omics, 'proteomics_omics_only_{}'.format(j), 
        #         overall_best_params[i]['exp2']['lr'], overall_best_params[i]['exp2']['lr_decay'],
        #         overall_best_params[i]['exp2']['batch_size'], overall_best_params[i]['exp2']['prot_hidden_dim'],
        #                          feature_types='metab', model_path='', fine_tune=False, seed=j,
        #                         hidden_dim=best_hidden_dim,num_layers=best_num_layers, dropout=best_dropout, return_preds=True)
        # results[i]['exp2'].append(val_auc)
        
#         print('experiment 3')
#         val_auc = run_experiment(transformer_input_sequences_omics, proteomics,
#             patient_indices, transformer_outcomes_omics, transformer_input_lengths_omics, 'both_omics_only_{}'.format(j), 
#                 overall_best_params[i]['exp3']['lr'], overall_best_params[i]['exp3']['lr_decay'],
#                 overall_best_params[i]['exp3']['batch_size'], overall_best_params[i]['exp3']['prot_hidden_dim'],
#                                  feature_types='both', model_path='', fine_tune=False, seed=j,
#                                 hidden_dim=overall_best_params[i]['exp3']['hidden_dim'],
#                                  num_layers=overall_best_params[i]['exp3']['num_layers'],
#                                  dropout=overall_best_params[i]['exp3']['dropout'],
#                                  return_preds=True, return_interpretability=True, return_grads=True)
#         results[i]['exp3'].append(val_auc)
        
#         print('experiment 4')
#         val_auc = run_experiment(transformer_input_sequences_omics, proteomics,
#             patient_indices, transformer_outcomes_omics, transformer_input_lengths_omics, 'EHR_PT_{}'.format(j), 
#                 overall_best_params[i]['exp4']['lr'], overall_best_params[i]['exp4']['lr_decay'],
#                 overall_best_params[i]['exp4']['batch_size'], feature_types='EHR',
#                                  model_path='./{}.pth'.format(best_model_name),
#                                  fine_tune=False, seed=j,
#                                 hidden_dim=best_hidden_dim,num_layers=best_num_layers,
#                                  dropout=best_dropout, return_preds=True)
#         results[i]['exp4'].append(val_auc)

        print('experiment 5')
        val_auc = run_experiment(transformer_input_sequences_omics, None,
            patient_indices_omics, transformer_outcomes_omics, transformer_input_lengths_omics, 'EHR_PT_FT_{}'.format(j), 
                overall_best_params[i]['exp5']['lr'], overall_best_params[i]['exp5']['lr_decay'],
                overall_best_params[i]['exp5']['batch_size'], feature_types='EHR',
                                 model_path='./models/predictive_models/{}.pth'.format(best_model_name),
                                 fine_tune=True, seed=j,
                                hidden_dim=best_hidden_dim,num_layers=best_num_layers,
                                 dropout=best_dropout, return_preds=True, vocab_size=len(merged_vocab),
                                 num_frozen=overall_best_params[i]['exp5']['num_frozen'])
        results[i]['exp5'].append(val_auc)

#         print('experiment 6')
#         val_auc = run_experiment(transformer_input_sequences_omics, proteomics,
#             patient_indices, transformer_outcomes_omics, transformer_input_lengths_omics, 'both_PT_FT_{}'.format(j), 
#                 overall_best_params[i]['exp6']['lr'], overall_best_params[i]['exp6']['lr_decay'],
#                 overall_best_params[i]['exp6']['batch_size'], overall_best_params[i]['exp6']['prot_hidden_dim'],
#                                  feature_types='both',
#                                 model_path='./{}.pth'.format(best_model_name),
#                                  fine_tune=True, seed=j,
#                                  hidden_dim=best_hidden_dim,num_layers=best_num_layers,
#                                  dropout=best_dropout,
#                                  return_preds=True, return_interpretability=True, return_grads=True)
#         results[i]['exp6'].append(val_auc)
    

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, average_precision_score, cohen_kappa_score

for exp in ['exp1','exp5']:
    true_outcomes = []
    total_preds = []
    indices = []
    for index_num, i in enumerate(results['MACE'][exp]):
        true_outcomes.extend(i[3])
        total_preds.extend(i[4])
        indices.extend(i[5])
    
    df = pd.DataFrame([true_outcomes, total_preds, indices]).T
    df.columns = ['true_outcome', 'pred', 'index']
    df = df.groupby('index').mean()
    
    print(exp)
    print('AUC: {:.3f}'.format(roc_auc_score(df['true_outcome'], df['pred'])))
    print('AUPRC: {:.3f}'.format(average_precision_score(df['true_outcome'], df['pred'])))
    
    epsilon = 1e-15
    df['pred'] = np.clip(df['pred'], epsilon, 1 - epsilon)
    
    # Calculating the BCELoss
    bce_loss = -np.mean(df['true_outcome'] * np.log(df['pred']) + (1 - df['true_outcome']) * np.log(1 - df['pred']))
    print('BCELoss: {:.3f}'.format(bce_loss))
    
    # Convert predictions to binary for Cohen's Kappa
    binary_preds = (df['pred'] > 0.5).astype(int)
    
    # Calculate Cohen's Kappa
    kappa = cohen_kappa_score(df['true_outcome'], binary_preds)
    print('Cohen\'s Kappa: {:.3f}'.format(kappa))
    print()

In [ ]:
with open('results.pkl', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)
